# 04 MS Semantic Kernel - Planner

## Intro 

## Step 1 - Read Environment / Create Semantic Kernel Instance

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"
#r "nuget: DotNetEnv, 2.5.0"

using Microsoft.SemanticKernel;
using DotNetEnv;

string _configurationFile = @"../01/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("AOAI_APIKEY");
string oAiEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT");
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("CHATCOMPLETION_DEPLOYMENTNAME");

IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        endpoint: oAiEndpoint, 
        apiKey: oAiApiKey,
        deploymentName: chatCompletionDeploymentName)
    .Build();

Console.WriteLine($"Semantic Kernel created...");


In [ ]:
using Microsoft.SemanticKernel;
using System.ComponentModel;
using System.Text.Json;
using System.Net.Http;

public class NativeFunctions {

    [SKFunction, Description("Download content from url")]
    public async Task<string> DownloadContentFromUrl(string url, int maxSize = 1000) 
    {
        using HttpClient httpClient = new HttpClient();
        
        HttpResponseMessage httpResponseMessage = await httpClient.GetAsync(url);
        JsonDocument jsonDocument = JsonDocument.Parse(await httpResponseMessage.Content.ReadAsStringAsync());

        JsonElement jsonElement = jsonDocument.RootElement; 
        return IterateJson(jsonElement).Substring(0, maxSize); 
    }

    string IterateJson(JsonElement root) 
    {
        string article = ""; 

        if (root.ValueKind == JsonValueKind.Array)
        {
            foreach (JsonElement element in root.EnumerateArray())
            {
                IterateJson(element);
            }
        }
        else if (root.ValueKind == JsonValueKind.Object)
        {
            foreach (JsonProperty property in root.EnumerateObject())
            {
                if (property.Name == "extract"){
                    article = property.Value.GetString()??"";
                    break; 
                }
                article = IterateJson(property.Value);
            }
        }
        return article; 
    }
}

Console.WriteLine($"Native function 'DownloadContentFromUrl' defined...");

In [ ]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using System.IO;

//Import native Function
string plugInName = "NativeFunctions";
NativeFunctions nativeFunctions = new NativeFunctions();
kernel.ImportFunctions(nativeFunctions, plugInName);

//Create semantic function (inline)
string skPrompt = @"Summarize the provided unstructured text in 3 easy to understand sentences. 
                        The sentences need to be short and provide the most important content of the provided text.
                        Text to summarize: {{$input}}";

OpenAIRequestSettings openAIRequestSettings = new OpenAIRequestSettings(){
            MaxTokens = 2000, 
            Temperature = 0.7, 
            TopP = 0.1
        };

kernel.CreateSemanticFunction(
    promptTemplate: skPrompt, 
    functionName: "SimpleSummarization",
    pluginName: "TextPlugin", 
    requestSettings: openAIRequestSettings
);

//Import semantic function (file)
string pluginsDirectory = Path.Combine(Directory.GetCurrentDirectory(), "Plugins");
plugInName = "TextPlugin";
kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, plugInName);

Console.WriteLine($"Inline function, semantic inline function, semantic function from file registered...");


In [ ]:
using Microsoft.SemanticKernel.Planners;
using Microsoft.SemanticKernel.Orchestration;

string actionDefinition = "Download a web page from a given url, summarize the content from the web page and extract the most important keywords from the summarization";

SequentialPlanner sequentialPlanner = new SequentialPlanner(kernel);        
string url = "https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Champions%20League&explaintext=1&exsectionformat=plain";

var customPlan = await sequentialPlanner.CreatePlanAsync(actionDefinition);
KernelResult kernelResult = await kernel.RunAsync(url, customPlan);

Console.WriteLine($"Extracted keywords: {kernelResult.GetValue<string>()}");
